In [4]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [3]:
tweets = pd.read_csv('Checkpoint2.csv')

# Train Validation Data Split

In [20]:

X_train, X_val, y_train, y_val = train_test_split(tweets.drop(['id','keyword','location','target'],axis=1), tweets[['target']], test_size=0.2, stratify=tweets[['target']], random_state=0)
X_train_text = X_train['text']
X_val_text = X_val['text']

print('X_train shape: ', X_train.shape)
print('X_val shape: ', X_val.shape)
print('y_train shape: ', y_train.shape)
print('y_val shape: ', y_val.shape)

X_train shape:  (6090, 12)
X_val shape:  (1523, 12)
y_train shape:  (6090, 1)
y_val shape:  (1523, 1)


In [21]:
print('Train Class Proportion:\n', y_train['target'].value_counts() / len(y_train) * 100)
print('\nValidation Class Proportion:\n', y_val['target'].value_counts() / len(y_val) * 100)

Train Class Proportion:
 0    57.027915
1    42.972085
Name: target, dtype: float64

Validation Class Proportion:
 0    57.058437
1    42.941563
Name: target, dtype: float64


<a id='Tokenization'></a>
## 6.1. Tokenization

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [23]:
tokenizer_1 = Tokenizer(num_words=5000, oov_token='<UNK>')
tokenizer_1.fit_on_texts(X_train_text)

In [24]:
X_train_text = tokenizer_1.texts_to_sequences(X_train_text)
X_val_text = tokenizer_1.texts_to_sequences(X_val_text)
print(X_train_text[:10])
print('')
print(X_val_text[:10])

[[59, 2, 4, 33, 5, 3, 1, 88, 1128, 360, 20, 998, 18, 14, 18, 15, 1423], [1, 2342, 2343, 3622, 39, 122, 117, 2, 1314, 235, 299], [3623, 339, 1, 191, 1, 1047, 80, 105], [8, 126, 9, 2, 1, 55, 1315, 881, 5, 33, 1, 435, 40, 1048, 1, 26, 12, 4, 1424, 1, 1983, 244, 10, 75, 1, 179], [9, 1, 58, 3624, 2344, 1, 499, 20, 1, 8, 272, 149, 94, 113, 8, 163, 2345, 153, 2, 716, 150], [1, 255, 7, 3, 2806], [13, 3, 164, 7, 1, 1425, 175, 2807, 11, 607, 3625, 2808], [132, 6, 561, 65, 216, 49, 81, 562, 132, 389, 21, 3, 201, 370, 7, 216, 637, 563], [134, 564, 53, 181, 10, 82, 61, 1049, 23, 350, 2809, 176, 10, 42, 118], [717, 310, 199, 23, 202, 1739, 224, 11, 1984, 7, 68, 638]]

[[3, 270, 1712, 9, 97, 11, 37, 3, 209, 1341, 31, 12, 6, 177, 196, 54, 7, 17, 294, 165, 4, 1340, 4972, 4973, 622, 270, 234], [10, 2, 3, 527, 1, 377, 17, 63, 558, 353, 9, 2, 1, 1], [1, 923, 774, 626, 2137, 4013], [787, 73, 21, 861, 22, 3, 862, 816, 39, 297, 659, 741, 62, 1022, 1079], [535, 204, 39, 236, 47, 116, 233, 93, 1184, 19, 158, 4

Each list in the `X_train_text` and `X_val_text` is a list of integers, which corresponds to each tweets in the train and validation set respectively. The length of each list is also different as different tweets have different lengths. Therefore, we will need to apply **padding** to make all sequences the same length.

We can use `tokenizer.word_index` to look at the vocabulary dictionary and `sequences_to_texts` to transform sequences back into texts. Note that words that are not in the vocabulary are now `<UNK>`.


**Note:** The Tokenizer stores everything in the `word_index` during `fit_on_texts`. Then, when calling the `texts_to_sequences` method, only the top `num_words` are considered. So `word_index` will actually contain more words than `num_words`.

In [25]:
tokenizer_1.sequences_to_texts([X_train_text[1]])

['<UNK> fest announce refund after day two be extreme weather evacuation']

<a id='Padding'></a>
## 6.2. Padding

After tokenization, each tweet is represented as a list of tokens. Next, we need to **pad** all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths). Do this by adding 0s to the end of each sentence in the tokenized form so that each sentence is *now the same length as the longest tweet*. 

The max length for the train set tweets is 32. We will set the `maxlen` to be 50 as tweets from the validation or test set might be longer. This means that texts longer than 50 words will be truncated to the 1st 50 words while texts shorter than 50 will have 0s appended to make them of length 50.

Below shows a quick example of padding sentences to a length of 5 sequences.

<br>

In [26]:
print('Train Set Max Length:', max(len(text) for text in X_train_text))
maxlen = 50

X_train_text = pad_sequences(X_train_text, padding='post', maxlen=maxlen)
X_val_text = pad_sequences(X_val_text, padding='post', maxlen=maxlen)

print('X_train shape:', X_train_text.shape)
print('X_train shape:', X_val_text.shape)

Train Set Max Length: 34
X_train shape: (6090, 50)
X_train shape: (1523, 50)


<a id='E_Matrix'></a>
## 6.3. Embedding Matrix – GloVe

We will use the [GloVe embeddings](https://nlp.stanford.edu/projects/glove/) that were pre-trained on 2 billion tweets to create our feature matrix. First, we will create a dictionary that will contain words as keys and their corresponding embedding list at values. The length of the embedding for each word will be 200, as the GloVe embedding we are using was trained to have 200 dimensions. Refer to [here](https://github.com/stanfordnlp/GloVe) also for more details.



In [30]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer_1.word_index) + 1

# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.twitter.27B.200d.txt', "r", encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193515 word vectors.


Next we will create an embedding matrix for our train vocab/corpus where each row number will correspond to the index of the word in our train vocab/corpus. The matrix will have 200 columns, each containing one of the GloVe feature/dimension.

In [31]:
# create a weight matrix for words in training set
embedding_matrix = np.zeros((vocab_size, 200))

for word, i in tokenizer_1.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
print('Embedding Matrix Shape:', embedding_matrix.shape)

Embedding Matrix Shape: (12727, 200)


<a id='Model_Build'></a>
# 7. Model Building & Training

<a id='LSTM'></a>
## 7.1. Long Short-Term Memory (LSTM)

[Long Short-Term Memory (LSTM)](https://www.mitpressjournals.org/doi/abs/10.1162/neco.1997.9.8.1735) models are a type of recurrent neural network that allows for longer range dependencies, unlike traditional feed-forward RNNs. It has a few advantages:

1. Longer range dependence
2. Selectively remember or forget things
3. Get around exploding and vanishing gradients

LSTMs have a few dependencies. Imagine that we are predicting whether it will rain today. This depends on several information:

1. The trend of the previous few days, such as many days with rain, or a very heavy downpour (the previous cell state)
2. Information from the previous day, such as temperature, wind level (previous hidden state)
3. Information from today (input at current time step)

To decide whether or not to use these information, LSTMs contain different memory blocks called **cells**, which are responsible for remembering which information are important to use and which to discard. Manipulations or changes to this memory is done through **gates**. 

1. The **forget gate** is responsible for removing memories from the cell state. From the figure and equations below, it takes inputs from the previous hidden state, `a` and current input `x`, and applies a sigmoid function to decide whether to keep the information or not. 
2. The **input gate** consists of the **update gate** and the **tanh** function. The process of adding new information to the memory cell is done through this gate. The **update gate** decides which information to be added through a sigmoid function (similar to the forget gate), and the tanh function creates the information to be added. The two outputs are then multiplied together and added to the memory cell.
3. The **output gate** selects useful information from the current cell state and outputs it. First, it creates a vector after applying **tanh** to the memory cell, then makes a filter using sigmoid function to regulate the information that needs to be used from the previous vector, and multiply them together, thus creating the output and also the hidden state to the next cell.



<img src='https://bn1301files.storage.live.com/y4moAJV3tGM4StMGVxvRmKYHz14V8F5X2aC0T4WaJdO1M_9QAPti5-3hx69bd-KJRsASCCdYErxqDL9PeNoDkFRFCwJnzpnR3e9w24NFJoOCMj3h_7jG90QjADEDje9hXVGM4sg8ltWrcbi2vz8pCLVBYsCTAQchMBn-JRTsX5ArSXY2r8ah54G_SVTJD9oJQOA?width=1711&height=623&cropmode=none' width=1000>

In [32]:
## Hyperparameters
num_epochs=15
dropout=0.2
recurrent_dropout=0.2
lr=0.0005
batch_size=128
class_weight = {0: y_train['target'].value_counts()[1]/len(y_train), 1: y_train['target'].value_counts()[0]/len(y_train)} 

We will use dropout and recurrent dropout to add regularization to the model, which can help with overfitting. Regular dropout works in the vertical direction of the RNN, while recurrent dropout masks the connections between the recurrent units (horizontal direction). Refer to this [post](https://stackoverflow.com/questions/44924690/keras-the-difference-between-lstm-dropout-and-lstm-recurrent-dropout) for more information.

A class weight will also be used. Without it, the model makes a lot more false negatives than false positives. The weighting for the minority class (`disaster`) will be given more weighting, meaning that it will be given more contribution  to the loss computation. This is taken as `(total samples-samples of class) / total samples`.

In [34]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, LSTM
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.callbacks import ModelCheckpoint

lstm_model = Sequential()
embedding_layer = Embedding(vocab_size, 200, weights=[embedding_matrix], input_length=maxlen, trainable=False)
lstm_model.add(embedding_layer)
lstm_model.add(LSTM(128, return_sequences=True, dropout=dropout, recurrent_dropout=recurrent_dropout)) # try adding dropout later
lstm_model.add(LSTM(128))

#model.add(Flatten())
lstm_model.add(Dense(1, activation='sigmoid'))

adam = optimizers.Adam(lr=lr)
lstm_model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])
print(lstm_model.summary())

# best hyperparameters
# num_epochs=15
# dropout=0.2
# recurrent_dropout=0.2
# lr=0.0005
# batch_size=128

ModuleNotFoundError: No module named 'keras.layers.core'

In [ ]:
def plot_model_performance(history):   
    plt.figure(figsize=(15,5))
    plt.plot(range(num_epochs), history.history['acc'],'-o',
             label='Train ACC',color='#ff7f0e')
    plt.plot(range(num_epochs),history.history['val_acc'],'-o',
             label='Val ACC',color='#1f77b4')
    x = np.argmax( history.history['val_acc'] ); y = np.max( history.history['val_acc'] )
    xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=200,color='#1f77b4')
    plt.text(x-0.03*xdist,y-0.13*ydist,'max acc\n%.2f'%y,size=14)
    plt.ylabel('Accuracy',size=14); plt.xlabel('Epoch',size=14)
    plt.legend(loc=(0.01,0.75))

    plt2 = plt.gca().twinx()
    plt2.plot(range(num_epochs),history.history['loss'],'-o',
              label='Train Loss',color='#2ca02c')
    plt2.plot(range(num_epochs),history.history['val_loss'],'-o',
              label='Val Loss',color='#d62728')
    x = np.argmin( history.history['val_loss'] ); y = np.min( history.history['val_loss'] )
    ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=200,color='#d62728')
    plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
   # plt.ylim([-0.2, 2])
    plt.ylabel('Loss',size=14)
    plt.xticks(ticks=list(range(num_epochs)),labels=list(range(1, num_epochs+1)))
    plt.legend(loc='lower left', bbox_to_anchor=(0.01, 0.1))
    plt.show()

In [ ]:
checkpoint = ModelCheckpoint('lstm_model.h5', monitor='val_acc', save_best_only=True)
history = lstm_model.fit(X_train_text, y_train, batch_size=batch_size, callbacks=[checkpoint], epochs=num_epochs, 
                         class_weight=class_weight, validation_data=(X_val_text, y_val), verbose=1)
plot_model_performance(history)

<a id='Attention'></a>
## 7.2. Bidirectional LSTM with Attention

<img src = 'https://bn1301files.storage.live.com/y4mpYVhDp9W6iW73-HkwwbyvkDRtQBj8K6FIz4kb7-iQhcydjC0KzXrREYJy-Im10aox7hLJIetYLNhuusOdo6fBkgpSLnnZn2RCf2H-lfqw1CXfsXUv_wFiuf2QAK70HgeNo_Ayl3H4kIbT5FUgCLK0iS21B5uNIAgFXVKapAYwMdMYzmStGqSBkvQ_H4m_9A6?width=850&height=425&cropmode=none' width=800>

A vanilla LSTM only uses information from the previous timesteps and not from the future. In many NLP problems, words that come after the current timepoint also influences the current output, although this is less likely for other applications like weather forecasting. As such, a bidirectional LSTM takes into account information from both past and future to create the output at the current timepoint, as shown by the figure above in the LSTM layer. Note that a gated recurrent unit (GRU) can also be used instead of a LSTM.

Also, another limitation with encoder-decoder architectures is that the encoder has to learn to encode input sequences into a *fixed-length internal representation*, which limits the performance of these networks, especially when considering very long input sequences. This means that the encoder has to compress all the information of a source input into a fixed-length vector and pass it to the encoder. The idea of **attention** aims to search for "a set of positions in a source sentence where the most relevant information is concentrated. The model then predicts a target word based on the context vectors associated with these source positions and all the previous generated target words." — [Bahdanau et al., 2015](https://arxiv.org/abs/1409.0473)

As seen from the figure above, the attention layer takes the bidirection hidden layer states and multiply them to a set of attention weights, which tells how much attention the current input should be paying attention to other past and future inputs (the **context**). These outputs at each timepoint will then be concatenated, which is the context, and will be used to generate the output. There are 2 main kinds of attention: **Global** and **Local** Attention.
* **Global Attention**: Considers all hidden states of encoder LSTM and all hidden states[(Luong et al., 2015)](https://arxiv.org/abs/1508.04025) / previous hidden states [(Bahdanau et al., 2015)](https://arxiv.org/abs/1409.0473) of the unidirectional encoder LSTM. Global attention requires lots of computation as all hidden states are considered.
* **Local Attention**: Only a part of the encoder hidden states are considered for context vector generation.




In [ ]:
## Attention Class

from keras.layers import Layer
import keras.backend as K

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
### Attention

## Hyperparameters
num_epochs=15
dropout=0.3
recurrent_dropout=0.3
lr=0.0005
batch_size=128

import tensorflow as tf
from keras.models import Sequential
from keras import Model
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, Input, Layer, GlobalMaxPooling1D, LSTM, Bidirectional, Concatenate
from keras.layers.embeddings import Embedding
from keras import optimizers

## Embedding Layer
sequence_input = Input(shape=(maxlen,))
embedded_sequences = Embedding(vocab_size, 200, weights=[embedding_matrix], trainable=False)(sequence_input)

## RNN Layer
lstm = Bidirectional(LSTM(128, return_sequences = True, dropout=dropout, recurrent_dropout=recurrent_dropout))(embedded_sequences)
# Getting our LSTM outputs
(lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(128, return_sequences=True, return_state=True))(lstm)

## Attention Layer
att_out=attention()(lstm)
outputs=Dense(1,activation='sigmoid')(att_out)
model_attn = Model(sequence_input, outputs)

adam = optimizers.Adam(lr=lr)
#sgd = optimizers.sgd(lr=lr)
model_attn.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])

print(model_attn.summary())

In [ ]:
checkpoint = ModelCheckpoint('attn_model.h5', monitor='val_acc', save_best_only=True)
history_attn = model_attn.fit(X_train_text, y_train, batch_size=batch_size, callbacks=[checkpoint], epochs=num_epochs, 
                              class_weight=class_weight, validation_data=(X_val_text, y_val), verbose=1)
plot_model_performance(history_attn)

<a id='BERT'></a>
## 7.3. BERT

The Bidirectional Encoder Representations from Transformers (BERT) is a language model developed by Google which has achieved state-of-the-art results in a variety of NLP tasks. BERT's key innovation is applying bidirectional training (actually it is non-directional, as it reads the entire sequence of words at once) of the encoder part of a **Transformer**.

To understand BERT, we need to first understand what is a Transformer. The Transformer was first introduced in the very influential paper "Attention is All You Need" by [Vaswani et al., 2017](https://arxiv.org/abs/1706.03762). Below is the architecture of the Transformer. It gets past the sequential nature of traditional RNNs, and instead considers all inputs at the same time using **multi-headed self-attention**. To get a better intuition of self-attention as well as a more detailed explanation of the Transformer, refer to this [post](https://www.analyticsvidhya.com/blog/2019/06/understanding-transformers-nlp-state-of-the-art-models/). Also, since the model is no longer sequential (contains no recurrence), it uses positional encodings to "inject some information about the relative or absolute position of the tokens in the sequence". These positional encodings use sine and cosine functions and are added to the input embeddings at the bottom of the encoder and decoder.

<img src = 'https://cdn.analyticsvidhya.com/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-19-53-10.png' align='left'>
<img src = 'https://cdn.analyticsvidhya.com/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-20-05-30.png'>


The BERT model uses a multi-layer bidirectional Transformer encoder (stacks the encoder several times). Only the encoder is needed as its goal is to create a language model. It performs self-attention in both directions and is pre-trained using two unsupervised prediction tasks.

**Masked Language Modelling** <br>
15% of the words in each sequence are masked at random, and the model was trained to predict these masked words based on the context provided by the other non-masked words in the sequence. The loss function only takes into consideration the prediction of the masked values and not the non-masked words.

**Next Sentence Prediction** <br>
BERT was also pre-trained to capture the relationships between consecutive sentences. It uses pairs of sentences as its training data. 5o% of the inputs are a pair in which the second sentence is the subsequent sentence in the original document while the other half is a random sentence from the corpus.

The figure below (upper) shows how BERT takes in the input and applies masking and sentence separation. The goal of training BERT is to minimize the combined loss function of these 2 strategies. Refer to this [post](https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270) for a more detailed explanation.

For single sentence classification such as the current problem of classifying disaster tweet, the architecture of BERT will involve adding a classification layer (sigmoid) on top of the Transformer output for the [CLS] token (lower graph below).

<br><br>

<img src = 'https://bn1301files.storage.live.com/y4mGNBqhZEX0ARXkCSvNbAkqw5PNaxxm_STcxiBcYvZVJLhdhjaNWbmnxbhxZwxyhJzPG6B7mjWwCQEWdLKJTtM9e7Z_A2y58uKJi2HoNyaU9wB4y9L66TXdp8UVvUSNPpDJc3XBGEld4gESOXDwZRc4xSYWuG_T7a5t8lDYQ3veqOCeBgt9N3IO6tI_PxaXZJV?width=1425&height=451&cropmode=none' width=600>
<img src = 'https://media.geeksforgeeks.org/wp-content/uploads/20200422012400/Single-Sentence-Classification-Task.png' width=500>

In [ ]:
# Hyperparameters
maxlen = 160
lr = 1e-5 # 1e-5 
num_epochs = 3 # 5
batch_size=16 # batch size cannot be too big for bert

The following code for building the BERT model was taken from [Wojtek Rosa's notebook](https://www.kaggle.com/wrrosa/keras-bert-using-tfhub-modified-train-data). Credit goes to him for sharing it.

In [ ]:
# We will use the official tokenization script created by the Google team
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tokenization


def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=512, lr=1e-5):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=lr), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
train_input = bert_encode(X_train.text.values, tokenizer, max_len=maxlen)
val_input = bert_encode(X_val.text.values, tokenizer, max_len=maxlen)
train_labels = y_train.target.values
val_labels = y_val.target.values

In [ ]:
bert_model = build_model(bert_layer, max_len=maxlen, lr=lr)
bert_model.summary()

In [ ]:
checkpoint = ModelCheckpoint('bertmodel.h5', monitor='val_accuracy', save_best_only=True)

bert_history = bert_model.fit(
    train_input, train_labels,
    validation_data=(val_input, val_labels),
    epochs=num_epochs,
    callbacks=[checkpoint], 
    #class_weight=class_weight,
    batch_size=batch_size
)

### RoBERTa

https://colab.research.google.com/github/DhavalTaunk08/NLP_scripts/blob/master/sentiment_analysis_using_roberta.ipynb#scrollTo=c3Q9NDdmqEyo

<a id='Meta-data'></a>
## 7.4. Feature-based Model

Here, we will create a feature-based model using the meta-features that we created at the beginning. The idea is to ensemble this model and the sequence models together to get better predictions. When ensembling, the outputs of this model will be given less weight compared to the neural networks as the neural networks are more likely to be better learners.

In [ ]:
X_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(n_estimators=500, max_depth=15, min_samples_split=20, min_samples_leaf=2, n_jobs=-1, random_state=0)
clf.fit(X_train.drop('text',axis=1), y_train.target.values)
clf_pred = clf.predict_proba(X_val.drop('text',axis=1))

print('Validation Accuracy:', accuracy_score(y_val.target.values, clf_pred.argmax(axis=-1)))

In [ ]:
clf_pred.max(axis=-1)

In [ ]:
clf_pred.max(axis=-1)*0.1

<a id='Error'></a>
# 8. Error Analysis

In [ ]:
# val = X_val.copy()
# val = val[['text']]
# val['target'] = y_val
# val['pred'] = model.predict(X_val_text)
# val['pred'] = (val['pred']*0.8) + (clf_pred.max(axis=-1)*0.2)
# val['pred'] = val['pred'].apply(lambda x: 1 if x >=0.5 else 0)
# error = val[val['target'] != val['pred']]
# error.head()

bert_model.load_weights('bertmodel.h5')
val = X_val.copy()
val = val[['text']]
val['target'] = y_val
# val['pred'] = lstm_model.predict_classes(X_val_text)
val['pred'] = bert_model.predict(val_input)
val['pred'] = val['pred'].apply(lambda x: 1 if x >=0.5 else 0)
error = val[val['target'] != val['pred']]
error.head()

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

# Plot confusion matrix
cm  = confusion_matrix(val.target, val.pred)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8),cmap=plt.cm.Blues)
plt.xticks(range(2), ['Non-Disaster', 'Disaster'], fontsize=16)
plt.yticks(range(2), ['Non-Disaster', 'Disaster'], fontsize=16)
plt.xlabel('Predicted Label',fontsize=18)
plt.ylabel('True Label',fontsize=18)
plt.show()

print('Num False Negatives:',sum((val['target'] == 1) & (val['pred'] == 0)))
print('Num False Positives:',sum((val['target'] == 0) & (val['pred'] == 1)))

There appears to be more false negatives than false positives from the validation data, meaning that more tweets are being labelled as `not disaster` when in fact they are, even after using `class_weights` to adjust for the imbalance. Perhaps `disaster` tweets can be given even more weighting depending on the goal/purpose of the classification.

In [ ]:
for t in error[(error['target'] == 1) & (error['pred'] == 0)]['text'].sample(n=20, random_state=0):
    print(t)

<a id='Test'></a>
# 9. Testing

In [ ]:
# count number of characters in each tweet
test['char_len'] = test.text.str.len()

# count number of words in each tweet
word_tokens = [len(word_tokenize(tweet)) for tweet in test.text]
test['word_len'] = word_tokens

# count number of sentence in each tweet
sent_tokens = [len(sent_tokenize(tweet)) for tweet in test.text]
test['sent_len'] = sent_tokens

In [ ]:
# polarity and subjectivity
test['polarity'] = [TextBlob(tweet).sentiment.polarity for tweet in test.text]
test['subjectivity'] = [TextBlob(tweet).sentiment.subjectivity for tweet in test.text]

#############################################################################################################################
# exclaimation and question marks
test['exclaimation_num'] = [tweet.count('!') for tweet in test.text]
test['questionmark_num'] = [tweet.count('?') for tweet in test.text]

#############################################################################################################################
# count number of hashtags and mentions
# Function for counting number of hashtags and mentions
def count_url_hashtag_mention(text):
    urls_num = len(re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text))
    word_tokens = text.split()
    hash_num = len([word for word in word_tokens if word[0] == '#' and word.count('#') == 1]) # only appears once in front of word 
    mention_num = len([word for word in word_tokens if word[0] == '@' and word.count('@') == 1]) # only appears once in front of word 
    return urls_num, hash_num, mention_num

url_num, hash_num, mention_num = zip(*[count_url_hashtag_mention(tweet) for tweet in test.text])
test = test.assign(url_num = url_num, hash_num = hash_num, mention_num = mention_num)

#############################################################################################################################
# count number of contractions
contractions = ["'t", "'re", "'s", "'d", "'ll", "'ve", "'m"]
test['contraction_num'] = [sum([tweet.count(cont) for cont in contractions]) for tweet in test.text]

In [ ]:
## Replace NaNs with 'None'
test.keyword.fillna('None', inplace=True) 

#############################################################################################################################
## Expand Contractions

# Function for expanding most common contractions https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
def decontraction(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

test.text = [decontraction(tweet) for tweet in test.text]

#############################################################################################################################
## Remove Emojis

# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

print(remove_emoji("OMG there is a volcano eruption!!! 😭😱😷"))

test.text = test.text.apply(lambda x: remove_emoji(x))

In [ ]:
#############################################################################################################################
## Remove URLs
test.text = test.text.apply(lambda x: remove_url(x))

#############################################################################################################################
## Remove Punctuations except '!?'

def remove_punct(text):
    new_punct = re.sub('\ |\!|\?', '', punctuation)
    table=str.maketrans('','',new_punct)
    return text.translate(table)

test.text = test.text.apply(lambda x: remove_punct(x))

#############################################################################################################################
## Replace amp
def replace_amp(text):
    text = re.sub(r" amp ", " and ", text)
    return text

test.text = test.text.apply(lambda x: replace_amp(x))

#############################################################################################################################

In [ ]:
# from wordsegment import load, segment
# load()

# test.text = test.text.apply(lambda x: ' '.join(segment(x)))

test = pd.read_csv('../input/twitter-logo/tweets_test_segmented.csv')

In [ ]:
## Lemmatization

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemma(text):
    words = word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(w.lower(), pos='v') for w in words])

test.text = test.text.apply(lambda x: lemma(x))

In [ ]:
# tokenize
test_text = test['text']
test_text = tokenizer_1.texts_to_sequences(test_text)

# padding
test_text = pad_sequences(test_text, padding='post', maxlen=50)

print('X_test shape:', test_text.shape)

In [ ]:
# lstm prediction
# model.predict(test_text)
lstm_model.load_weights('lstm_model.h5')
submission = test.copy()[['id']]
submission['target'] = lstm_model.predict_classes(test_text)
submission.to_csv('submission.csv', index=False)
display(submission.head())



In [ ]:
# bi-lstm attention prediction
model_attn.load_weights('attn_model.h5')
submission_attn = test.copy()[['id']]
submission_attn['target'] = model_attn.predict(test_text)
submission_attn['target'] = submission_attn['target'].apply(lambda x: 1 if x >=0.5 else 0)
submission_attn.to_csv('submission_attn.csv', index=False)
display(submission_attn.head())

In [ ]:
# bert prediction

test_input = bert_encode(test.text.values, tokenizer, max_len=160)

bert_model.load_weights('bertmodel.h5')
submission_bert = test.copy()[['id']]
submission_bert['target'] = bert_model.predict(test_input)
submission_bert['target'] = submission_bert['target'].apply(lambda x: 1 if x >=0.5 else 0)
submission_bert.to_csv('submission_bert.csv', index=False)
display(submission_bert.head())

In [ ]:
# bert + meta-features prediction

clf_testpred = clf.predict_proba(test.drop(['id','keyword','location','text'],axis=1))
submission_bert = test.copy()[['id']]
submission_bert['target'] = (bert_model.predict(test_input)*0.8).ravel() + (clf_testpred.max(axis=1)*0.2)
submission_bert['target'] = submission_bert['target'].apply(lambda x: 1 if x >=0.5 else 0)
submission_bert.to_csv('submission_bert_ensemble.csv', index=False)
display(submission_bert.head())

In [ ]:
submission_bert['target'].plot(kind='hist')

<a id='Conclusion'></a>
# 10. Conclusion

https://stackabuse.com/python-for-nlp-movie-sentiment-analysis-using-deep-learning-in-keras/

https://www.analyticsvidhya.com/blog/2020/03/pretrained-word-embeddings-nlp/

### attention
https://matthewmcateer.me/blog/getting-started-with-attention-for-classification/


##  TO DO
* Word and Char vectorizer
* Remove numbers? Convert numbers to words?
* Unigrams, Bigrams and Trigrams
* Glove; remove stopwords, clean before glove?
* Logistic Regression, BOW, TD IDF, GloVe, BERT?
* Check Duplicates
* Decaying LR